In [17]:
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import f1_score
from time import time

In [2]:
  mnist = tf.contrib.learn.datasets.load_dataset("mnist")

Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:
Please use tf.data.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [7]:
X_tr = mnist.train.images
y_tr = mnist.train.labels[:, np.newaxis]
enc = OneHotEncoder()
y_tr = enc.fit_transform(y_tr).toarray()
X_te = mnist.test.images
y_te = mnist.test.labels[:, np.newaxis]
y_te = enc.transform(y_te).toarray()


n_ftrs = X_tr.shape[1]

X_tr = X_tr.reshape(-1, 28, 28, 1)
X_te = X_te.reshape(-1, 28, 28, 1)

In [14]:
run=0

In [15]:
run += 1
tf.reset_default_graph()
with tf.device('/cpu:0'):
    X = tf.placeholder(X_tr.dtype, (None, 28, 28, 1), 'attributes')
    y = tf.placeholder(y_tr.dtype, (None, 10), 'labels')
    filters = tf.get_variable('kernel', (5,5,1,1), X_tr.dtype, tf.random_normal_initializer(.0, .1))
    CL = tf.layers.conv2d(X, 16, [5,5], 1, padding='SAME')
#     CL = tf.nn.conv2d(input=X, filter=filters, strides=[1, 1, 1, 1], padding='SAME')
    MP = tf.contrib.layers.max_pool2d(CL, [2,2])
    pool_flatttened = tf.reshape(MP, [-1,16*14*14*1])
    D = tf.layers.dense(pool_flatttened, units=100, activation=tf.nn.relu)
    logits = tf.layers.dense(D, units=10)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y))
    optimiser = tf.train.GradientDescentOptimizer(learning_rate=.1).minimize(loss)
    tf.summary.scalar('loss', loss)
    inference = tf.nn.softmax(logits)
writer = tf.summary.FileWriter('data/.graph/run_{}'.format(run), tf.get_default_graph())
merged_smr = tf.summary.merge_all()


In [20]:
epochs = 10
batch_size = 500
c_time = time()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(epochs):
        
        for b in range(X_tr.shape[0] // batch_size):
            
            sess.run(optimiser, feed_dict={X: X_tr[b*batch_size : (b+1)*batch_size], y: y_tr[b*batch_size : (b+1)*batch_size]})
        smr = sess.run(merged_smr, feed_dict={X: X_tr[i*batch_size : (i+1)*batch_size], y: y_tr[i*batch_size : (i+1)*batch_size]})
        writer.add_summary(smr, i)
    prd = sess.run(inference, feed_dict={X:X_te})
writer.close()
print('Time taken =', time()-c_time)

Time taken = 188.75807213783264


In [13]:
f1_score(y_te.argmax(axis=1), prd.argmax(axis=1), average='weighted')

0.9425067531178242